Assessing one minute

In [4]:
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import logging
import sys
sys.path.append("..")

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from finbert.finbert import *
import finbert.utils as tools

import nltk 
nltk.download("punkt")

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', None)

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Thais\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
lm_path = 'models/language_model/finbertTRC2'
cl_path = 'models/classifier_model/finbert-sentiment'
cl_data_path = 'data/sentiment_data'

Modelo

In [7]:
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')
model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert', output_attentions = True)
label_list = ['positive', 'neutral', 'negative']

02/07/2023 22:05:14 - INFO - filelock -   Lock 2396499834248 acquired on C:\Users\Thais/.cache\huggingface\hub\models--ProsusAI--finbert\blobs\e15a7b5738df7f17553399b6d94c6e2ff69c89245d066e8e5d183f5803a554e3.lock
Downloading: 100%|██████████| 438M/438M [01:31<00:00, 4.80MB/s] 
02/07/2023 22:06:46 - INFO - filelock -   Lock 2396499834248 released on C:\Users\Thais/.cache\huggingface\hub\models--ProsusAI--finbert\blobs\e15a7b5738df7f17553399b6d94c6e2ff69c89245d066e8e5d183f5803a554e3.lock


Texto Teste

In [17]:
#Positive +Negative
exert0 = """After the reporting period , BioTie North American licensing partner Somaxon Pharmaceuticals announced positive results with nalmefene in a pilot Phase 2 clinical trial for smoking cessation

            They will cover all Forest Industry's units and employees in Finland and concern temporary and permanent lay-offs"""

#Positive
exert1 = "After the reporting period , BioTie North American licensing partner Somaxon Pharmaceuticals announced positive results with nalmefene in a pilot Phase 2 clinical trial for smoking cessation"

#Negative
exert2 = "They will cover all Forest Industry 's units and employees in Finland and concern temporary and permanent lay-offs"

#Neutral
exert3 = "A few employees would remain at the Oulu plant in support functions for other group units" 

Funções

In [8]:
def get_split(text_input):

    l_total = []
    l_parcial = []

    if len(text_input.split())//150 > 0:
        n = len(text_input.split())//150
    else:
        n = 1
    
    for w in range(n):
        if w == 0:
            l_parcial = text_input.split()[:200]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text_input.split()[w*150:w*150 + 200]
            l_total.append(" ".join(l_parcial))
    return l_total

def change_range(df, min_v, max_v):
    lst = []
    for i in range(0, len(df)):
        old_value = df[i]
        old_min = min_v
        old_max = max_v
        new_min = 0
        new_max = 1
    
        new_value = ((old_value - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min
        lst += [new_value]
    
    return lst  


Análise de Sentimento

In [18]:
teste = exert0

textos = pd.DataFrame(teste.split('\n\n'), columns = ['Text'])
textos['Result'] = ""

for d in range(0, len(textos['Text'])):
    result = predict(textos['Text'][d], model)
    result['prob_pos'] = result.logit.apply(lambda x:x[0])
    textos['Result'][d] = result.prob_pos.mean() 


02/07/2023 22:28:08 - INFO - root -   Using device: cpu 
02/07/2023 22:28:08 - INFO - finbert.utils -   *** Example ***
02/07/2023 22:28:08 - INFO - finbert.utils -   guid: 0
02/07/2023 22:28:08 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
02/07/2023 22:28:08 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
02/07/2023 22:28:08 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
02/07/2023 22:28:08 - INFO - finbert.utils -   tok

Normalização e Análise

In [19]:
textos

,Text,Result
0,"After the reporting period , BioTie North American licensing partner Somaxon Pharmaceuticals announced positive results with nalmefene in a pilot Phase 2 clinical trial for smoking cessation",0.944006
1,They will cover all Forest Industry's units and employees in Finland and concern temporary and permanent lay-offs,0.006902


In [16]:
min_v = textos['Result'].min()
max_v = textos['Result'].max()
textos['Scores_adj'] = np.array(change_range(textos['Result'], min_v, max_v))

textos['Scores_adj'].mean()*100

50.0